# Phase 1: Data Preparation with Local PySpark

Cleans the raw Zomato CSV and writes a Delta Lake table to `../delta_lake/raw/restaurants`.

The SparkSession config is identical to what you'd use on Databricks — only `.master("local[*]")` is added for local execution.

In [44]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws, trim, regexp_replace, when

## 1. Start SparkSession with Delta Lake support

In [45]:
spark = (
    SparkSession.builder
    .appName("ZomatoSemanticSearch-DataPrep")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .master("local[*]")   # Remove this line when running on Databricks
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")
print(f"Spark version: {spark.version}")

Spark version: 3.5.3


## 2. Load raw CSV

In [46]:
RAW_CSV   = "../data/zomato_with_dynamic_menu.csv"
DELTA_OUT = "../delta_lake/raw/restaurants"

df_raw = spark.read.csv(RAW_CSV, header=True, inferSchema=True)
print(f"Rows: {df_raw.count():,}  |  Columns: {len(df_raw.columns)}")
df_raw.printSchema()

Rows: 9,557  |  Columns: 22
root
 |-- Restaurant ID: string (nullable = true)
 |-- Restaurant Name: string (nullable = true)
 |-- Country Code: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Locality: string (nullable = true)
 |-- Locality Verbose: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Cuisines: string (nullable = true)
 |-- Average Cost for two: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Has Table booking: string (nullable = true)
 |-- Has Online delivery: string (nullable = true)
 |-- Is delivering now: string (nullable = true)
 |-- Switch to order menu: string (nullable = true)
 |-- Price range: string (nullable = true)
 |-- Aggregate rating: string (nullable = true)
 |-- Rating color: string (nullable = true)
 |-- Rating text: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- menu: string (nullable = true)



In [47]:
df_raw.show(5, truncate=False)

+-------------+----------------------+------------+----------------+-----------------------------------------------------------------------+------------------------------------------+------------------------------------------------------------+----------+---------+--------------------------------+--------------------+----------------+-----------------+-------------------+-----------------+--------------------+-----------+----------------+------------+-----------+-----+----------------------------------------------------------------------------------------+
|Restaurant ID|Restaurant Name       |Country Code|City            |Address                                                                |Locality                                  |Locality Verbose                                            |Longitude |Latitude |Cuisines                        |Average Cost for two|Currency        |Has Table booking|Has Online delivery|Is delivering now|Switch to order menu|Price range|Aggregate ra

## 3. Clean & enrich

- Standardise column names to snake_case
- Strip whitespace, commas from `rate`
- Build `text_for_embedding` — the combined field the model will encode

In [48]:
# Rename columns to a clean, consistent schema
# Matches the global Zomato dataset (Restaurant ID, Restaurant Name, Aggregate rating, etc.)
rename_map = {
    "Restaurant Name":        "name",
    "City":                   "location",
    "Address":                "address",
    "Locality":               "locality",
    "Cuisines":               "cuisines",
    "Average Cost for two":   "cost_for_two",
    "Has Table booking":      "book_table",
    "Has Online delivery":    "online_order",
    "Aggregate rating":       "rating",
    "Votes":                  "votes",
    "Price range":            "price_range",
    "Currency":               "currency",
    "Country Code":           "country_code",
}

df = df_raw
for old, new in rename_map.items():
    if old in df.columns:
        df = df.withColumnRenamed(old, new)

df.printSchema()

root
 |-- Restaurant ID: string (nullable = true)
 |-- name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- location: string (nullable = true)
 |-- address: string (nullable = true)
 |-- locality: string (nullable = true)
 |-- Locality Verbose: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- cuisines: string (nullable = true)
 |-- cost_for_two: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- book_table: string (nullable = true)
 |-- online_order: string (nullable = true)
 |-- Is delivering now: string (nullable = true)
 |-- Switch to order menu: string (nullable = true)
 |-- price_range: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- Rating color: string (nullable = true)
 |-- Rating text: string (nullable = true)
 |-- votes: string (nullable = true)
 |-- menu: string (nullable = true)



In [49]:
# Global dataset: rating is already float (0.0–5.0); treat 0.0 as unrated
df = df.withColumn(
    "rating",
    when(col("rating").cast("float") == 0.0, None)
    .otherwise(col("rating").cast("float"))
)

# cost_for_two is already numeric — just cast to int
df = df.withColumn("cost_for_two", col("cost_for_two").cast("int"))

# Trim string fields
for c in ["name", "cuisines", "location", "locality"]:
    if c in df.columns:
        df = df.withColumn(c, trim(col(c)))

# Clean menu: "['Butter Chicken', 'Dal Makhani']" → "Butter Chicken, Dal Makhani"
if "menu" in df.columns:
    df = df.withColumn(
        "menu",
        trim(regexp_replace(col("menu"), r"[\[\]']", ""))
    )

# Drop rows with no name or cuisines
df = df.dropna(subset=["name", "cuisines"])

print(f"Clean rows: {df.count():,}")
df.select("name", "cuisines", "menu").show(5, truncate=False)

Clean rows: 9,542
+----------------------+--------------------------------+----------------------------------------------------------------------------+
|name                  |cuisines                        |menu                                                                        |
+----------------------+--------------------------------+----------------------------------------------------------------------------+
|Le Petit Souffle      |French, Japanese, Desserts      |Chef Special, Signature Dish, Popular Combo, House Dessert, Beverage Special|
|Izakaya Kikufuji      |Japanese                        |Chef Special, Signature Dish, Popular Combo, House Dessert, Beverage Special|
|Heat - Edsa Shangri-La|Seafood, Asian, Filipino, Indian|Chef Special, Signature Dish, Popular Combo, House Dessert, Beverage Special|
|Ooma                  |Japanese, Sushi                 |Chef Special, Signature Dish, Popular Combo, House Dessert, Beverage Special|
|Sambo Kojin           |Japanese, Kor

In [50]:
from pyspark.sql.functions import when as sql_when

price_label = (
    sql_when(col("price_range") == 1, "budget friendly affordable casual")
    .when(col("price_range") == 2, "mid-range moderate comfortable")
    .when(col("price_range") == 3, "upscale fine dining premium")
    .when(col("price_range") == 4, "luxury high-end exclusive")
    .otherwise("")
)

# Include menu items — this is the key signal for dish-level queries like "bajji", "vada", "biryani"
embed_parts = [col("name"), col("cuisines"), price_label, col("location")]
if "menu" in df.columns:
    embed_parts.append(col("menu"))

df = df.withColumn("text_for_embedding", concat_ws(" | ", *embed_parts))

df.select("name", "cuisines", "menu", "text_for_embedding").show(5, truncate=False)

+----------------------+--------------------------------+----------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|name                  |cuisines                        |menu                                                                        |text_for_embedding                                                                                                                                                                     |
+----------------------+--------------------------------+----------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Le Petit Souffle      |French, Japanese, D

## 4. Add a surrogate restaurant_id

In [51]:
from pyspark.sql.functions import monotonically_increasing_id

df = df.withColumn("restaurant_id", monotonically_increasing_id().cast("string"))

keep_cols = [
    "restaurant_id", "name", "cuisines", "location", "locality",
    "address", "rating", "votes", "cost_for_two", "price_range",
    "currency", "country_code", "book_table", "online_order",
    "menu", "text_for_embedding",
]
df = df.select([c for c in keep_cols if c in df.columns])
print(df.columns)

['restaurant_id', 'name', 'cuisines', 'location', 'locality', 'address', 'rating', 'votes', 'cost_for_two', 'price_range', 'currency', 'country_code', 'book_table', 'online_order', 'menu', 'text_for_embedding']


## 5. Write to Delta Lake

In [52]:
(
    df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .save(DELTA_OUT)
)

print(f"Saved to {DELTA_OUT}")

Saved to ../delta_lake/raw/restaurants


## 6. Verify

In [53]:
df_check = spark.read.format("delta").load(DELTA_OUT)
print(f"Delta rows: {df_check.count():,}")
df_check.select("restaurant_id", "name", "rating", "text_for_embedding").show(5, truncate=False)

Delta rows: 9,542
+-------------+----------------------+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|restaurant_id|name                  |rating|text_for_embedding                                                                                                                                                                     |
+-------------+----------------------+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0            |Le Petit Souffle      |4.8   |Le Petit Souffle | French, Japanese, Desserts | upscale fine dining premium | Makati City | Chef Special, Signature Dish, Popular Combo, House Dessert, Beverage Special               |
|1            |Izakaya Kikufuji      |4.5   |Izakaya Kikufuji 

In [54]:
spark.stop()